In [13]:
!pip install panda -q
!pip install tensorflow -q
!pip install keras -q
!pip install numpy -q
!pip install matplotlib -q
!pip install seaborn -q

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning models
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam


In [15]:
df = pd.read_csv('bank-additional-full.csv', sep=';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [16]:
# Informações sobre as features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [17]:
# Estatísticas acerca das variáveis numéricas
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


In [18]:
# Checando a ocorrência de missing data
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [19]:
# Verificar valores únicos de cada coluna categórica
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 
                    'loan', 'contact', 'month', 'day_of_week',"y","poutcome"]

# Criar uma cópia para não modificar o original
df_encoded = df.copy()


# Label Encoding 

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [20]:
# ============================================
# 4. SEPARAÇÃO DE FEATURES E TARGET
# ============================================
# AJUSTE AQUI: substitua 'y' pelo nome da sua coluna target
X = df_encoded.drop('y', axis=1)
y = df_encoded['y']

# Se o target for categórico, fazer encoding
if y.dtype == 'object':
    le_target = LabelEncoder()
    y = le_target.fit_transform(y)
    print(f"\nClasses do target: {le_target.classes_}")

print("\n" + "=" * 50)
print("SEPARAÇÃO FEATURES E TARGET")
print("=" * 50)
print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nDistribuição do target:")
print(pd.Series(y).value_counts())
print(f"Proporção de classes: {pd.Series(y).value_counts(normalize=True)}")


SEPARAÇÃO FEATURES E TARGET
Shape de X: (41188, 20)
Shape de y: (41188,)

Distribuição do target:
y
0    36548
1     4640
Name: count, dtype: int64
Proporção de classes: y
0    0.887346
1    0.112654
Name: proportion, dtype: float64


In [21]:
# 5. DIVISÃO TREINO/VALIDAÇÃO/TESTE
# ============================================
# Primeiro: separar treino+validação e teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.15,
    random_state=42,
    stratify=y
)

# Segundo: separar treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.176,  # ~15% do total
    random_state=42,
    stratify=y_temp
)

print("\n" + "=" * 50)
print("DIVISÃO DOS DADOS")
print("=" * 50)
print(f"Treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validação: {X_val.shape[0]} amostras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(X)*100:.1f}%)")


DIVISÃO DOS DADOS
Treino: 28847 amostras (70.0%)
Validação: 6162 amostras (15.0%)
Teste: 6179 amostras (15.0%)


In [22]:
# NORMALIZAÇÃO/PADRONIZAÇÃO

print("\n" + "=" * 50)
print("NORMALIZANDO OS DADOS")
print("=" * 50)

scaler = StandardScaler()

# Fit apenas no treino e transform em todos
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✓ Normalização concluída")
print(f"Range dos valores de treino: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")


NORMALIZANDO OS DADOS
✓ Normalização concluída
Range dos valores de treino: [-5.11, 19.57]


In [23]:
# 3. MODELO 

print("\n" + "=" * 50)
print("CONSTRUINDO MODELO OTIMIZADO")
print("=" * 50)

# Modelo MENOR e MAIS RÁPIDO
model = Sequential([
    # Camada 1: Menos neurônios = mais rápido
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),
    BatchNormalization(),  # Acelera convergência
    Dropout(0.3),
    
    # Camada 2: Ainda menor
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    # Saída
    Dense(1, activation='sigmoid')
])

# Otimizador com learning rate maior = convergência mais rápida
optimizer = Adam(learning_rate=0.001)  # Padrão, mas pode aumentar para 0.003

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())


CONSTRUINDO MODELO OTIMIZADO


/opt/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 32)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,409 (5.50 KB)

 Trainable params: 1,313 (5.13 KB)

 Non-trainable params: 96 (384.00 B)

None


In [24]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Reduzido de 15 para 5
    restore_best_weights=True,
    verbose=1
)

# Reduz learning rate se estabilizar
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)


In [ ]:
# 5. TREINAMENTO OTIMIZADO
# ============================================
print("\n" + "=" * 50)
print("TREINANDO MODELO (MODO RÁPIDO)")
print("=" * 50)

history = model.fit(
    X_train_scaled, y_train,
    epochs=50,  # Reduzido de 100 para 50
    batch_size=128,  # AUMENTADO de 32 para 128 (MUITO MAIS RÁPIDO!)
    validation_split=0.2,  # Validação interna (mais rápido que split separado)
    callbacks=[early_stop, reduce_lr],
    verbose=1  # Mostrar progresso
)



TREINANDO MODELO (MODO RÁPIDO)
Epoch 1/50


In [ ]:
# 10. AVALIAÇÃO DO MODELO
# ============================================
print("\n" + "=" * 50)
print("AVALIANDO O MODELO")
print("=" * 50)

# Avaliação no conjunto de teste
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'\n✓ Acurácia no teste: {accuracy:.4f} ({accuracy*100:.2f}%)')
print(f'✓ Loss no teste: {loss:.4f}')

# Fazer previsões
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

# Métricas detalhadas
print("\n" + "=" * 50)
print("RELATÓRIO DE CLASSIFICAÇÃO")
print("=" * 50)
print(classification_report(y_test, y_pred))

# AUC-ROC Score
try:
    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f"AUC-ROC Score: {auc_score:.4f}")
except:
    print("Não foi possível calcular AUC-ROC")

# Matriz de confusão
print("\n" + "=" * 50)
print("MATRIZ DE CONFUSÃO")
print("=" * 50)
cm = confusion_matrix(y_test, y_pred)
print(cm)

# ============================================
# 11. VISUALIZAÇÕES
# ============================================
print("\n" + "=" * 50)
print("GERANDO VISUALIZAÇÕES")
print("=" * 50)

# Plotar histórico de treinamento
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Acurácia
axes[0].plot(history.history['accuracy'], label='Treino')
axes[0].plot(history.history['val_accuracy'], label='Validação')
axes[0].set_title('Acurácia por Época')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Acurácia')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], label='Treino')
axes[1].plot(history.history['val_loss'], label='Validação')
axes[1].set_title('Loss por Época')
axes[1].set_xlabel('Época')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
print("✓ Gráfico de histórico salvo: training_history.png")

# Plotar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title('Matriz de Confusão')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predito')
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
print("✓ Matriz de confusão salva: confusion_matrix.png")

plt.show()

print("\n" + "=" * 50)
print("✓ PROCESSO COMPLETO FINALIZADO!")
print("=" * 50)
print("\nArquivos gerados:")
print("- best_model.keras (melhor modelo)")
print("- training_history.png (gráficos de treinamento)")
print("- confusion_matrix.png (matriz de confusão)")


In [ ]:

# 11. VISUALIZAÇÕES

print("\n" + "=" * 50)
print("GERANDO VISUALIZAÇÕES")
print("=" * 50)

# Plotar histórico de treinamento
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Acurácia
axes[0].plot(history.history['accuracy'], label='Treino')
axes[0].plot(history.history['val_accuracy'], label='Validação')
axes[0].set_title('Acurácia por Época')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Acurácia')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], label='Treino')
axes[1].plot(history.history['val_loss'], label='Validação')
axes[1].set_title('Loss por Época')
axes[1].set_xlabel('Época')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
print("✓ Gráfico de histórico salvo: training_history.png")

# Plotar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title('Matriz de Confusão')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predito')
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
print("✓ Matriz de confusão salva: confusion_matrix.png")

plt.show()

print("\n" + "=" * 50)
print("✓ PROCESSO COMPLETO FINALIZADO!")
print("=" * 50)
print("\nArquivos gerados:")
print("- best_model.keras (melhor modelo)")
print("- training_history.png (gráficos de treinamento)")
print("- confusion_matrix.png (matriz de confusão)")

Dicionário de Dados

- **age**: Idade do cliente em anos (numérico)

- **job**: Tipo de ocupação (categórico: admin., blue-collar, entrepreneur, housemaid, management, retired, self-employed, services, student, technician, unemployed)

- **marital**: Estado civil (categórico: divorced, married, single, unknown; nota: divorced inclui viúvos)

- **education**: Nível de escolaridade (categórico: basic.4y, basic.6y, basic.9y, high.school, illiterate, professional.course, university.degree)

- **default**: Possui crédito em default? (categórico: no, yes)

- **housing**: Possui empréstimo imobiliário? (categórico: no, yes)

- **loan**: Possui empréstimo pessoal? (categórico: no, yes)

- **contact**: Tipo de comunicação do contato (categórico: cellular, telephone)

- **month**: Mês do último contato (categórico: jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec)

- **day_of_week**: Dia da semana do último contato (categórico: mon, tue, wed, thu, fri)

- **duration**: Duração do último contato em segundos (numérico)

- **campaign**: Número de contatos realizados durante esta campanha (numérico, inclui último contato)

- **pdays**: Dias desde o último contato da campanha anterior (numérico; 999 = não foi contactado anteriormente)

- **previous**: Número de contatos antes desta campanha (numérico)

- **poutcome**: Resultado da campanha anterior (categórico: failure, nonexistent, success)

- **emp.var.rate**: Taxa de variação do emprego - indicador trimestral (numérico)

- **cons.price.idx**: Índice de preços ao consumidor - indicador mensal (numérico)

- **cons.conf.idx**: Índice de confiança do consumidor - indicador mensal (numérico)

- **euribor3m**: Taxa Euribor de 3 meses - indicador diário (numérico)

- **nr.employed**: Número de empregados - indicador trimestral (numérico)

- **y**: O cliente subscreveu um depósito a prazo? (binário: yes, no)